In [21]:
import xarray as xr
import numpy as np
import tqdm as tq

from toolbar.filter import MovingAverageFilter
from toolbar.masked import masked   # 气象工具函数


In [22]:
# 数据读取
data_year = ['1961', '2022']
# 读取CN05.1逐日最高气温数据
CN051_1 = xr.open_dataset(r"E:\data\CN05.1\1961_2021\CN05.1_Tmax_1961_2021_daily_025x025.nc")
CN051_2 = xr.open_dataset(r"E:\data\CN05.1\2022\CN05.1_Tmax_2022_daily_025x025.nc")
Tmax = xr.concat([CN051_1, CN051_2], dim='time')
Tmax = masked(Tmax, r"D:\PyFile\map\地图边界数据\长江区1：25万界线数据集（2002年）\长江区.shp")  # 掩膜处理得长江流域温度
Tmax = Tmax.sel(time=Tmax['time.month'].isin([6, 7, 8])).groupby('time.year')  # 截取夏季数据

C:\Users\86136\.conda\envs\Meteorological\lib\site-packages\xarray\coding\times.py:154: SerializationWarning: Ambiguous reference date string: 1-1-1 00:00:00. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0001-1-1 00:00:00). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)


In [23]:
Tmax_5Day_filt = np.array([[[MovingAverageFilter(iyear[1]['tmax'].data[:, i, j], 'lowpass', [5], np.nan).filted() for j in range(283)] for i in range(163)] for iyear in tq.tqdm(Tmax)])  # 5天滑动平均
Tmax_5Day_filt = Tmax_5Day_filt.transpose(0, 3, 1, 2) # 转换为(year, day, lat, lon)格式

100%|██████████| 62/62 [02:02<00:00,  1.98s/it]


In [28]:
Tmax_5Day_filt = xr.DataArray(Tmax_5Day_filt,
                                coords=[[str(i) for i in range(eval(data_year[0]), eval(data_year[1]) + 1)],
                                        [str(i) for i in range(1, 88 + 1)],
                                        CN051_2['lat'].data,
                                        CN051_2['lon'].data],
                                dims=['year', 'day', 'lat', 'lon'],)
Tmax_5Day_filt.to_netcdf(fr"D:\PyFile\p2\data\Tmax_5Day_filt.nc")

In [34]:
zone_stations = masked((CN051_2-CN051_2+1).sel(time='2022-01-01'), r"D:\PyFile\map\地图边界数据\长江区1：25万界线数据集（2002年）\长江区.shp") xr.open_dataset(r"E:\data\CN05.1\2022\CN05.1_Tmax_2022_daily_025x025.nc")

27.26026611328125

In [ ]:
#返回40百分位值, 忽略nan值
np.nanpercentile(Tmax_5Day_filt, 50)